# 5年营业收入增长率 sales_growth
过去5个财年的 每股营业收入增长 除以 年均每股营业收入

注意：对于基本面数据，
每年11-第二年4月使用前一年0930的TTM数据，既三季报+前一年四季度
5-8月使用1季度0331的TTM数据，
9-10月使用中报0630的TTM数据，
11-12月使用当年三季报0930的TTM数据

In [14]:
import sys

# sys.path.append('J:\\Quant\\freq_used_funcs')
print(sys.path)

['D:\\program\\PyCharm 2022.3.3\\plugins\\python\\helpers-pro\\jupyter_debug', 'D:\\program\\PyCharm 2022.3.3\\plugins\\python\\helpers\\pydev', 'J:\\Quant\\JQ\\scap_growth', 'J:\\Quant\\JQ\\scap_growth', 'D:\\program\\python310.zip', 'D:\\program\\DLLs', 'D:\\program\\lib', 'D:\\program', 'J:\\Quant\\junhe_quant\\scap_growth', '', 'J:\\Quant\\junhe_quant\\scap_growth\\lib\\site-packages', 'J:\\Quant\\junhe_quant\\scap_growth\\lib\\site-packages\\win32', 'J:\\Quant\\junhe_quant\\scap_growth\\lib\\site-packages\\win32\\lib', 'J:\\Quant\\junhe_quant\\scap_growth\\lib\\site-packages\\Pythonwin', 'J:\\Quant\\freq_used_funcs']


In [19]:
# 获取tushare每股营业收入数据
import tushare as ts

pro = ts.pro_api('e9628663b45f87ea92e39aaa7127063830ebd5090bc1e8943138b84f')
import security_basic_info




In [20]:
df = pro.fina_indicator_vip(period='20221231')
df

,ts_code,ann_date,end_date,eps,dt_eps,total_revenue_ps,revenue_ps,capital_rese_ps,surplus_rese_ps,undist_profit_ps,...,roe_yoy,bps_yoy,assets_yoy,eqt_yoy,tr_yoy,or_yoy,q_sales_yoy,q_op_qoq,equity_yoy,update_flag
0,A22231.SZ,20230118,20221231,4.14,4.05,27.3891,27.3891,1.4895,0.5000,6.4809,...,62.8621,74.3120,62.2231,74.3135,58.6078,58.6078,NaN,NaN,74.3135,1
1,A22460.SZ,20230118,20221231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.1659,NaN,45.6121,25.5637,91.0482,91.0482,NaN,NaN,25.5637,1
2,A22278.SZ,20230130,20221231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-11.0430,NaN,30.7480,41.9701,21.4938,21.4938,NaN,NaN,41.9701,0
3,A22137.BJ,20230201,20221231,0.45,0.45,13.0174,13.0174,1.6935,0.1906,0.8117,...,4.2923,4.7087,-1.5606,4.7099,15.8528,15.8528,40.7736,477.0235,4.7099,1
4,A21410.SZ,20230310,20221231,0.27,0.27,2.6092,2.6092,0.2330,0.0900,0.4380,...,43.0436,17.3300,4.2451,17.7526,36.9964,36.9964,3.2901,41.0361,17.7526,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,300602.SZ,20230328,20221231,0.19,0.19,8.1200,8.1200,1.4673,0.2645,2.2642,...,206.7988,3.8571,13.4031,4.1800,34.8757,34.8757,27.1002,112.9481,4.1800,1
952,000728.SZ,20230328,20221231,0.40,0.40,1.2240,1.2240,3.8979,0.3965,1.4172,...,-11.0162,1.9942,12.9670,1.9900,-12.5816,-12.5816,-38.5017,23.8076,1.9940,1
953,301387.SZ,20230328,20221231,1.99,1.99,17.4685,17.4685,3.7484,0.3324,4.9683,...,-26.4285,19.6679,19.7587,19.6675,0.0342,0.0342,NaN,NaN,19.6675,1
954,688669.SH,20230328,20221231,0.48,0.48,42.4009,42.4009,9.4933,0.5000,5.5041,...,-47.5376,3.3609,13.9227,3.3610,55.6982,55.6982,8.4631,391.3618,3.3610,1


In [22]:
# 获取指定报告期所有证券的财务指标
date = '20230326'
securities = security_basic_info.get_all_security(date=date, result_type='ts_code')

df = pro.fina_indicator_vip(period='20221231')
revenue_ps_df = df.loc[:, ['ts_code', 'ann_date', 'end_date', 'eps', 'revenue_ps']]
revenue_ps_df.dropna(inplace=True)
revenue_ps_df = revenue_ps_df.loc[revenue_ps_df['ts_code'].isin(securities)]
revenue_ps_df = revenue_ps_df[revenue_ps_df['ann_date'] <= date]
revenue_ps_df = revenue_ps_df.reset_index()
revenue_ps_df.drop('index', axis=1)
revenue_ps_df

,index,ts_code,ann_date,end_date,eps,revenue_ps
0,10,001311.SZ,20230206,20221231,4.2000,31.6510
1,11,001225.SZ,20230206,20221231,4.5500,10.2397
2,13,001311.SZ,20230206,20221231,4.2000,31.6510
3,16,001225.SZ,20230206,20221231,4.5500,10.2397
4,27,301059.SZ,20230209,20221231,0.5400,2.2973
...,...,...,...,...,...,...
233,803,000990.SZ,20230325,20221231,0.0435,9.6420
234,804,002712.SZ,20230325,20221231,-0.7039,7.6904
235,805,001337.SZ,20230325,20221231,0.5500,1.3124
236,807,002465.SZ,20230325,20221231,0.2900,2.4369


In [74]:
# 获取tusahre财务指标数据，可指定数据品种、证券范围以及数据获取时间
def get_fin_indicators(date, period, indicators, security=[], market_sign='A'):
    '''
    :param security: security list
    :param date: yyyymmdd
    :param period: yyyymmdd
    :param indicators: list
    :return: dataframe
    '''

    fields = ['ts_code', 'ann_date', 'end_date', 'update_flag']
    fields.extend(indicators)
    revenue_ps_df = pro.fina_indicator_vip(period=period, fields=fields)
    revenue_ps_df.dropna(inplace=True)
    # 查看是否指定了证券区间
    if len(security) > 0:
        revenue_ps_df = revenue_ps_df.loc[revenue_ps_df['ts_code'].isin(security)]
    if market_sign == 'A':
        revenue_ps_df = revenue_ps_df[revenue_ps_df['ts_code'].str.startswith(('0','3','6'))]
    revenue_ps_df = revenue_ps_df[revenue_ps_df['ann_date'] <= date]
    revenue_ps_df = revenue_ps_df.reset_index()
    revenue_ps_df.drop('index', axis=1)
    return revenue_ps_df


get_fin_indicators(date='20230316', period='20221231', indicators=['eps', 'revenue_ps'])

,index,ts_code,ann_date,end_date,eps,revenue_ps,update_flag
0,10,001311.SZ,20230206,20221231,4.2000,31.6510,0
1,11,001225.SZ,20230206,20221231,4.5500,10.2397,0
2,12,603073.SH,20230206,20221231,1.2600,8.4560,0
3,13,001311.SZ,20230206,20221231,4.2000,31.6510,1
4,16,001225.SZ,20230206,20221231,4.5500,10.2397,1
...,...,...,...,...,...,...,...
200,413,002179.SZ,20230316,20221231,1.7249,9.7106,1
201,414,688535.SH,20230316,20221231,0.6800,5.0106,1
202,419,301203.SZ,20230316,20221231,2.4800,6.2738,1
203,709,603291.SH,20230216,20221231,0.3400,3.0375,1


In [68]:


# 拉取数据
df = pro.stock_basic(**{
    "ts_code": "",
    "name": "",
    "exchange": "",
    "market": "",
    "is_hs": "",
    "list_status": "",
    "limit": "",
    "offset": ""
}, fields=[
    "ts_code",
    "symbol",
    "name",
    "area",
    "industry",
    "market",
    "list_date"
])
print(df)



        ts_code  symbol     name  area industry market list_date
0     000001.SZ  000001     平安银行    深圳       银行     主板  19910403
1     000002.SZ  000002      万科A    深圳     全国地产     主板  19910129
2     000004.SZ  000004     ST国华    深圳     软件服务     主板  19910114
3     000005.SZ  000005     ST星源    深圳     环境保护     主板  19901210
4     000006.SZ  000006     深振业A    深圳     区域地产     主板  19920427
...         ...     ...      ...   ...      ...    ...       ...
5121  873223.BJ  873223     荣亿精密  None     None    北交所  20220609
5122  873305.BJ  873305     九菱科技  None     None    北交所  20221221
5123  873339.BJ  873339     恒太照明  None     None    北交所  20221117
5124  873527.BJ  873527      夜光明  None     None    北交所  20221027
5125  689009.SH  689009  九号公司-WD    北京      摩托车    CDR  20201029

[5126 rows x 7 columns]


D:\program\PyCharm 2022.3.3\jbr\bin


In [62]:

from datetime import datetime


def get_dates(n_years):
    today = datetime.today()

    dates = []
    for i in range(0, n_years + 2):
        year = today.year - i
        dates.append(datetime(year, 3, 31).strftime('%Y%m%d'))
        dates.append(datetime(year, 6, 30).strftime('%Y%m%d'))
        dates.append(datetime(year, 9, 30).strftime('%Y%m%d'))
        dates.append(datetime(year, 12, 31).strftime('%Y%m%d'))
    dates.sort()
    dates = [x for x in dates if x < today.strftime('%Y%m%d')][n_years * 4 * (-1):]
    return dates


period_list = get_dates(n_years=5)

In [64]:
len(period_list)

20

In [65]:
# 根据报告期列表获取单季数据,这里指单季营业收入同比增速
date = datetime.today().strftime('%Y%M%D')
get_fin_indicators(date=date, period=period_list[0], indicators=['eps', 'revenue_ps'])

,index,ts_code,ann_date,end_date,eps,revenue_ps
0,1,000563.SZ,20180428,20180331,0.0396,0.0876
1,2,000022.SZ,20180427,20180331,0.2270,0.9550
2,5,600087.SH,20180424,20180331,0.0179,0.1680
3,6,000043.SZ,20180428,20180331,0.1013,2.3005
4,7,300571.SZ,20180426,20180331,0.6000,2.9230
...,...,...,...,...,...,...
3785,3956,832802.BJ,20180428,20180331,0.1100,0.6316
3786,3957,832149.BJ,20180425,20180331,0.0040,0.7147
3787,3958,834770.BJ,20180428,20180331,0.1400,1.2357
3788,3959,830809.BJ,20180427,20180331,0.1100,0.6671
